In [1]:
from sklearn.datasets import make_blobs
import pandas as pd
import plotly.express as px

import numpy as np

from sklearn.cluster import KMeans

import time

In [2]:
def EuclideanDistance(a, b):
    return np.linalg.norm(a-b)

def BruteForceSearch(Principal, TestPoint):
    MinEuclideanDistance = np.inf
    for i in range(len(Principal)):
        Dist = EuclideanDistance(Principal.loc[i, ['x1', 'x2', 'x3']], TestPoint)
        if Dist < MinEuclideanDistance:
            ClosePoint = i
            MinEuclideanDistance = Dist
    return Principal['PointName'][ClosePoint]

# Inicia os parâmetros para a geração da base de dados

In [3]:
#Initial Parameters
NumberOfData = 10000
NumberOfCenters = 5
PlotData = False

# Geração da base de dados

In [4]:
X, y = make_blobs(n_samples = NumberOfData, centers = NumberOfCenters, n_features=3, cluster_std = 0.08, center_box = [0, 1])
DataFrame = pd.DataFrame(X, columns = ['x1', 'x2', 'x3'])
DataFrame['y'] = y
DataFrame['PointName'] = DataFrame.index
DataFrame['PointName'] = DataFrame['PointName'].apply(lambda x: 'p' + str(x))

In [5]:
DataFrame[['x1', 'x2', 'x3']].describe().transpose()

,count,mean,std,min,25%,50%,75%,max
x1,10000.0,0.519151,0.251062,-0.136852,0.330150,0.559471,0.732464,1.075205
x2,10000.0,0.537008,0.385637,-0.129341,0.169411,0.438626,0.952511,1.231879
x3,10000.0,0.523057,0.214533,-0.028889,0.337159,0.491662,0.725545,1.053206


# Plot da base de dados gerada

In [6]:
if PlotData:
    fig = px.scatter_3d(DataFrame, x="x1", y="x2", z="x3", color = 'y')
    fig.show()

# Remoção da variável de output dos dados gerados

In [7]:
DataFrame = DataFrame.drop(['y'], axis = 1)

# Declaração dos parâmetros do algoritmo K-Means

In [8]:
NClusters = 4

In [9]:
kmeans = KMeans(n_clusters=NClusters).fit(DataFrame[['x1', 'x2', 'x3']])
DataFrame['KMeans'] = kmeans.labels_

In [10]:
if PlotData:
    fig = px.scatter_3d(DataFrame, x="x1", y="x2", z="x3", color = 'KMeans')
    fig.show()

In [11]:
DataFrame

,x1,x2,x3,PointName,KMeans
0,0.804405,0.538887,0.303149,p0,3
1,0.435434,0.180533,0.730449,p1,1
2,0.589036,1.104541,0.381682,p2,2
3,0.361288,0.242431,0.731942,p3,1
4,0.053772,1.016402,0.326104,p4,0
...,...,...,...,...,...
9995,0.593098,1.164956,0.667829,p9995,2
9996,0.409156,0.180244,0.832301,p9996,1
9997,0.686289,0.945746,0.411406,p9997,2
9998,0.095231,0.949888,0.248984,p9998,0


# Teste da metodologia

- Etapas Globais:
    - Determina-se a quantidade de testes a ser realizado;
    - Determina-se os parâmetros de teste;
    - Gera-se uma lista de pontos de teste aleatórios.
    
- Busca exaustiva:
    - Para cada ponto aleatório na lista de pontos aleatórios faz-se a busca do ponto mais próximo.

- Busca no cluster:
    - Para cada ponto aleatório na lista de pontos aleatórios encontra a qual cluster ele pertence pelo algoritmo K-Means;
    - Faz-se a busca do ponto mais próximo olhando os pontos da base principal que fazem parte do cluster que o ponto aleatório foi determinado.


In [12]:
NpointsOfTest = 10 # Quantidade de pontos de teste

In [13]:
PointsOfTest = []
for i in range(NpointsOfTest):
    PointsOfTest.append(list(np.random.rand(1,3)[0]))
PointsOfTest = pd.DataFrame(PointsOfTest, columns = ['x1', 'x2', 'x3'])

In [14]:
MatchedPoint = []
TimeVector = []
for PointNumber in range(NpointsOfTest):
    StartTime = time.time()
    MatchedPoint.append(BruteForceSearch(DataFrame, PointsOfTest.loc[PointNumber,['x1', 'x2', 'x3']]))
    TimeVector.append(time.time() - StartTime)
    
TimeDataFrame = pd.DataFrame(TimeVector, columns = ['TempoBruteForce'])
PointFound = pd.DataFrame(MatchedPoint, columns = ['BruteForcePoint'])

In [15]:
MatchedPoint = []
TimeVector = []
for PointNumber in range(NpointsOfTest):
    StartTime = time.time()
    TestPoint = np.array(PointsOfTest.loc[PointNumber,['x1', 'x2', 'x3']]).reshape(1, -1)
    ClusterFound = kmeans.predict(TestPoint)[0]
    MatchedPoint.append(BruteForceSearch(DataFrame[DataFrame['KMeans'] == ClusterFound].reset_index(drop = True), TestPoint.reshape(-1)))
    TimeVector.append(time.time() - StartTime)
    
TimeDataFrame['TempoCluster'] = TimeVector
PointFound['ClusterPoint'] = MatchedPoint

In [16]:
TimeDataFrame.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
TempoBruteForce,10.0,8.093645,0.479368,7.492295,7.731150,8.017166,8.358777,8.887616
TempoCluster,10.0,2.311183,0.779448,1.455678,1.548622,2.174803,3.039249,3.316097


In [17]:
PointFound

,BruteForcePoint,ClusterPoint
0,p5069,p5069
1,p6162,p6162
2,p2562,p2562
3,p4458,p4458
4,p3131,p3131
5,p1620,p1620
6,p1316,p1316
7,p5301,p5301
8,p6658,p6658
9,p2299,p2299
